In [1]:
# read a file called '/data/apkwan/class_pw.txt'
import os, pandas as pd
import pymysql
from sqlalchemy import create_engine, Column, MetaData
import clickhouse_driver
from clickhouse_sqlalchemy import (
    Table, make_session, get_declarative_base, types, engines
)
import getpass,time
from IPython.display import display, HTML
import yfinance as yf
# We can do things using DuckDB or MySQL or whatever

global_university=False

In [2]:
import os

credentials_path = 'data/credentials.txt'
user='communal' #ideally this would be in a file as well


if os.path.exists(credentials_path):
    with open(credentials_path) as f:
        # read the password
        password = f.readline().strip()
else:
    # enter it directly
    password = getpass.getpass()
    # write the password to the file
    with open(credentials_path, 'w') as f:
        f.write(password)


In [3]:
# https://alankwan.atlassian.net/wiki/spaces/PUB/pages/439714217/Ways+to+access+databases
def clickhouse_query(query, global_university = False):
    """
    :param query: the query string
    :return: the query result
    """
    # feed the password for this query
    # read a file that contains the password
    # report timings
    start_time = pd.Timestamp.now()
    # only works on the university network i think
    if global_university:
        db_con = clickhouse_driver.Client(host='chenlin01.fbe.hku.hk', port=9000, user=user,
                                        password=password,
                                            database='crsp',
                                            settings={'use_numpy': True})
    # If you are not on the university network, you can use the following
    else:
        print('using the localexpose reversel tunnel')
        db_con = clickhouse_driver.Client(host='ap.loclx.io', port=46749, user=user,
                        password=password,
                            database='crsp',
                            settings={'use_numpy': True})
                                          
    end_time = pd.Timestamp.now()
    print(f'Clickhouse query took {end_time - start_time}')
    df, types=db_con.execute(query,with_column_types=True)
    # convert df to dataframe
    df2=pd.DataFrame(df)
    # retrieve columns to the dataframe
    df2.columns=[x[0] for x in types]
    return df2

In [4]:
# show me databases
clickhouse_query('show databases')

using the localexpose reversel tunnel
Clickhouse query took 0 days 00:00:00.001241


,name
0,class_communal
1,common_goods
2,comp
3,comp_202501
4,contrib_202401
5,crsp
6,crsp_202501
7,factset_common_2024
8,factset_own_2024
9,fset_revere_debt


In [5]:
clickhouse_query('show tables from crsp')

using the localexpose reversel tunnel
Clickhouse query took 0 days 00:00:00


,name
0,acti
1,asia
2,asib
3,asic
4,asio
...,...
252,wrds_inddlytranspose_query
253,wrds_indmthtranspose_query
254,wrds_monthlyindexret_query
255,wrds_msfv2_query


In [6]:
# Fetch SPY 1 year data from CRSP database
# Assuming SPY ticker is in the CRSP database and we want the most recent 1 year of data

# Query to get SPY data for the last year
query = """
SELECT 
    permno,
    ticker
FROM 
    stocknames_v2
WHERE
    ticker='SPY'
LIMIT
    10
"""

# Execute the query
spy_data = clickhouse_query(query)




using the localexpose reversel tunnel
Clickhouse query took 0 days 00:00:00


In [8]:
# Download SPY data
spy = yf.Ticker("SPY")

# Get historical data for the last year
spy.history(period="max", interval = "1d")



,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
Date,,,,,,,,
1993-01-29 00:00:00-05:00,24.469914,24.469914,24.348173,24.452522,1003200,0.0,0.0,0.0
1993-02-01 00:00:00-05:00,24.469912,24.626436,24.469912,24.626436,480500,0.0,0.0,0.0
1993-02-02 00:00:00-05:00,24.609030,24.695988,24.556856,24.678596,201300,0.0,0.0,0.0
1993-02-03 00:00:00-05:00,24.713397,24.956879,24.696006,24.939487,529400,0.0,0.0,0.0
1993-02-04 00:00:00-05:00,25.026436,25.096002,24.748171,25.043827,531500,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2025-03-25 00:00:00-04:00,575.299988,576.409973,573.690002,575.460022,38355700,0.0,0.0,0.0
2025-03-26 00:00:00-04:00,575.190002,576.330017,567.190002,568.590027,51848300,0.0,0.0,0.0
2025-03-27 00:00:00-04:00,567.179993,570.900024,564.940002,567.080017,42164200,0.0,0.0,0.0
